In [13]:
import pandas as pd
import sys
import os
sys.path.append(os.path.dirname(sys.path[0]) + '/src')
from ast import literal_eval
from datasets.ie_hyperion_dataset import find_word_bounds, clean_text

df = pd.read_csv('../data/processed/splitted_union/ie_s2_hyperion_test.csv', converters={'Stralci': literal_eval, 'Repertori': literal_eval})
#df = df.head(50)
df['Testo'] = df['Testo'].map(clean_text)
df['Stralci'] = df['Stralci'].map(lambda x: [clean_text(s) for s in x])
df['Bounds'] = df.apply(lambda x: find_word_bounds(x['Stralci'], x['Testo']), axis=1).values.tolist()

In [15]:
df

,Testo,Stralci,Repertori,Bounds,Stralci_predetti,Bounds_predetti
0,"Comunque, se tutti i giornalisti che oggi si s...","[Comunque,, se tutti i giornalisti che oggi si...","[commento, valutazione, possibilità]","[(0, 0), (1, 36), (37, 41)]","[Comunque, se tutti i giornalisti che oggi si ...","[(0, 41)]"
1,Ha suscitato scandalo la proposta di Salvini d...,[Ha suscitato scandalo la proposta di Salvini ...,"[sancire, valutazione, conferma, conferma, imp...","[(0, 12), (13, 33), (34, 51), (52, 63), (64, 7...",[Ha suscitato scandalo la proposta di Salvini ...,"[(0, 12), (13, 33), (34, 51), (52, 63), (64, 7..."
2,Via via che si aggrava la crisi prodotta dalla...,[Via via che si aggrava la crisi prodotta dall...,"[causa, sancire, causa, sancire, giudizio, com...","[(0, 21), (22, 35), (36, 55), (56, 78), (79, 8...",[Via via che si aggrava la crisi prodotta dall...,"[(0, 21), (22, 55), (56, 78), (79, 160), (161,..."
3,"""In Veneto, livelli più elevati di tamponi e m...","[""In Veneto, livelli più elevati di tamponi e ...","[valutazione, sancire]","[(0, 32), (33, 53)]","[""In Veneto, livelli più elevati di tamponi e ...","[(0, 32), (33, 53)]"
4,"Presidente, il corretto di stile di vita è tor...","[Presidente, il corretto di stile di vita è to...","[opinione, sancire]","[(0, 15), (16, 30)]","[Presidente, il corretto di stile di vita è to...","[(0, 15), (16, 30)]"
...,...,...,...,...,...,...
1238,"Il fatto è che intanto è tutto fermo, e per mu...","[Il fatto è che intanto è tutto fermo, e per m...","[sancire, possibilità]","[(0, 20), (21, 53)]","[Il fatto è che intanto è tutto fermo, e per m...","[(0, 20), (21, 53)]"
1239,libero di pensare che sia banale influenza . I...,"[libero di pensare che sia banale influenza .,...","[sancire, specificazione]","[(0, 6), (7, 25)]","[libero di pensare che sia banale influenza .,...","[(0, 6), (7, 25)]"
1240,"Zaia ,lasciane qualcuno per i ritardatari se p...","[Zaia ,lasciane qualcuno per i ritardatari, se...","[sancire, specificazione, sancire, previsione,...","[(0, 5), (6, 13), (14, 15), (16, 24), (25, 31)]","[Zaia ,lasciane qualcuno per i ritardatari se ...","[(0, 31)]"
1241,ZAIA SERVONO VACCINI ASL 7 BASSANO DEL GRAPPA!...,[ZAIA SERVONO VACCINI ASL 7 BASSANO DEL GRAPPA...,"[sancire, prescrizione]","[(0, 7), (8, 19)]",[ZAIA SERVONO VACCINI ASL 7 BASSANO DEL GRAPPA...,"[(0, 7), (8, 19)]"


In [23]:
from models.nltk_segmenter import NLTKSegmenter
from trainers.bert_seg_trainer import *


nltk_seg = NLTKSegmenter()
df['Stralci_predetti'] = df['Testo'].map(nltk_seg.predict).values.tolist()
df['Bounds_predetti'] = df.apply(lambda x: find_word_bounds(x['Stralci_predetti'], x['Testo']), axis=1).values.tolist()

df['Bounds_predetti_norm'] = df.apply(lambda x: normalize_bounds_by_repertoire(x['Bounds_predetti'], x), axis=1).values.tolist()

In [18]:
df['Bounds_predetti'].iloc[0]

[(0, 41)]

In [21]:
from trainers.bert_seg_trainer import *
from datasets.ie_hyperion_dataset import IEHyperionDataset

test_dataset = IEHyperionDataset(df, 'dbmdz/bert-base-italian-xxl-uncased')

metrics = compute_metrics(df['Bounds_predetti'].to_list(), test_dataset)
metrics

{'windowdiff': 0.5377331087580638,
 'ghd': 69.07510917030568,
 'pk': 0.3048955724237861,
 'iou': 0.5091148174116299}

In [25]:
norm_metrics = compute_metrics(df['Bounds_predetti_norm'].to_list(), test_dataset)
norm_metrics

{'windowdiff': 0.32240403731538053,
 'ghd': 27.031441048034935,
 'pk': 0.20217541539007025,
 'iou': 0.878052741961476}